# Bài 1: Khôi phục Masked Token (Masked Language Modeling)

In [2]:
from transformers import pipeline

# 1. Tải pipeline "fill-mask" (tự động tải mô hình mặc định phù hợp, thường là BERT variant)
mask_filler = pipeline("fill-mask", model="bert-base-uncased")

# 2. Câu đầu vào với token [MASK]
input_sentence = "Hanoi is the [MASK] of Vietnam."

# 3. Thực hiện dự đoán, lấy 5 dự đoán hàng đầu (top_k=5)
predictions = mask_filler(input_sentence, top_k=5)

# 4. In kết quả
print(f"Câu gốc: {input_sentence}")
for pred in predictions:
    print(f"Dự đoán: '{pred['token_str']}' với độ tin cậy: {pred['score']:.4f}")
    print(f" -> Câu hoàn chỉnh: {pred['sequence']}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Câu gốc: Hanoi is the [MASK] of Vietnam.
Dự đoán: 'capital' với độ tin cậy: 0.9991
 -> Câu hoàn chỉnh: hanoi is the capital of vietnam.
Dự đoán: 'center' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the center of vietnam.
Dự đoán: 'birthplace' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the birthplace of vietnam.
Dự đoán: 'headquarters' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the headquarters of vietnam.
Dự đoán: 'city' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the city of vietnam.


# Bài 2: Dự đoán từ tiếp theo (Next Token Prediction)

In [5]:
from transformers import pipeline

# 1. Tải pipeline "text-generation" (tự động tải mô hình mặc định, thường là GPT-2)
generator = pipeline("text-generation")

# 2. Đoạn văn bản mồi (prompt)
prompt = "The best thing about learning NLP is"

# 3. Sinh văn bản (max_length=50, num_return_sequences=3)
# params:
#     max_length: tổng độ dài của câu mồi và phần được sinh ra
#     num_return_sequences: số lượng chuỗi kết quả muốn nhận
generated_texts = generator(prompt, max_length=50, num_return_sequences=1, truncation=True)

# 4. In kết quả
print(f"Câu mồi: '{prompt}'")
for text in generated_texts:
    print("Văn bản được sinh ra:")
    print(text['generated_text'])

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Câu mồi: 'The best thing about learning NLP is'
Văn bản được sinh ra:
The best thing about learning NLP is that it's easy and fast.

NLP is a simple, easy to learn, and highly practical, language for all skill levels. There are no specific requirements, so learning NLP is easy.

This guide is for those who have already learned NLP in class. It covers the basic concepts of NLP, as well as the more advanced concepts such as declarative, imperative, and a bit more.

To learn NLP, you need to understand the underlying concepts of C++, and learn the concepts of imperative, declarative, and a bit more.

NLP is best learned by using NST and NLP-C++.

NLP-C++ is a great, high quality language for C++.

So what is NLP?

NLP is a programming language suitable for C++.

I've read a lot of NLP books and articles about NLP, and I think this is a really good starting point for learning NLP. NLP is a very simple language, and can be used for many different purposes.

This is a very good starting poin

# Bài 3: Tính toán Vector biểu diễn của câu (Sentence Representation)

In [6]:
import torch
from transformers import AutoTokenizer, AutoModel

# 1. Chọn một mô hình BERT
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. Câu đầu vào
sentences = ["This is a sample sentence."]

# 3. Tokenize câu
# padding=True: đệm các câu ngắn hơn để có cùng độ dài
# truncation=True: cắt các câu dài hơn
# return_tensors='pt': trả về kết quả dưới dạng PyTorch tensors
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# 4. Đưa qua mô hình để lấy hidden states
# torch.no_grad() để không tính toán gradient, tiết kiệm bộ nhớ
with torch.no_grad():
    outputs = model(**inputs)

# outputs.last_hidden_state chứa vector đầu ra của tất cả các token
last_hidden_state = outputs.last_hidden_state
# shape: (batch_size, sequence_length, hidden_size)

# 5. Thực hiện Mean Pooling
# Để tính trung bình cộng của các vector, cần bỏ qua token đệm (padding tokens)
attention_mask = inputs['attention_mask']
mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

# Nhân các vector ẩn với mask và tính tổng (chỉ giữ lại vector của các token thực)
sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)

# Tính tổng của mask để chia trung bình (sử dụng min=1e-9 để tránh chia cho 0)
sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)

# Vector biểu diễn cuối cùng của câu
sentence_embedding = sum_embeddings / sum_mask

# 6. In kết quả
print("Vector biểu diễn của câu:")
print(sentence_embedding)
print("\nKích thước của vector:", sentence_embedding.shape)

Vector biểu diễn của câu:
tensor([[-6.3874e-02, -4.2837e-01, -6.6779e-02, -3.8430e-01, -6.5784e-02,
         -2.1826e-01,  4.7636e-01,  4.8659e-01,  4.0647e-05, -7.4273e-02,
         -7.4740e-02, -4.7635e-01, -1.9773e-01,  2.4824e-01, -1.2162e-01,
          1.6678e-01,  2.1045e-01, -1.4576e-01,  1.2636e-01,  1.8635e-02,
          2.4640e-01,  5.7090e-01, -4.7014e-01,  1.3782e-01,  7.3650e-01,
         -3.3808e-01, -5.0331e-02, -1.6452e-01, -4.3517e-01, -1.2900e-01,
          1.6516e-01,  3.4004e-01, -1.4930e-01,  2.2422e-02, -1.0488e-01,
         -5.1916e-01,  3.2964e-01, -2.2162e-01, -3.4206e-01,  1.1993e-01,
         -7.0148e-01, -2.3126e-01,  1.1224e-01,  1.2550e-01, -2.5191e-01,
         -4.6374e-01, -2.7261e-02, -2.8415e-01, -9.9249e-02, -3.7017e-02,
         -8.9192e-01,  2.5005e-01,  1.5816e-01,  2.2701e-01, -2.8497e-01,
          4.5300e-01,  5.0945e-03, -7.9441e-01, -3.1008e-01, -1.7403e-01,
          4.3029e-01,  1.6816e-01,  1.0590e-01, -4.8987e-01,  3.1856e-01,
          3.